# Base OpenAI Client

```python
from openai import OpenAI, AzureOpenAI
client = OpenAI(
    base_url="http://localhost:8080/openai/v1",
    api_key="test",
)
```

In [33]:
from openai import OpenAI, AzureOpenAI

client = OpenAI(
    base_url="http://localhost:8080/openai/v1",
    api_key="test",
)

print(client.models.list())

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="llama3.2:latest",
    stream=False,
)

print(chat_completion.choices[0].message.content)

SyncPage[Model](data=[Model(id='llama3.2:latest', created=1731215929, object='model', owned_by='library')], object='list')
It appears to be one. How can I assist you today?


In [29]:
### Streaming
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Write me a pretty short story",
        }
    ],
    model="llama3.2:latest",
    stream=True,
)

for i in chat_completion:
    print(i.choices[0].delta.content, end="")
print()

NotFoundError: Error code: 404 - {'error': {'message': 'model "llama3" not found, try pulling it first', 'type': 'api_error', 'param': None, 'code': None}}

### Azure OpenAI Client

```python
client = AzureOpenAI(
    base_url="http://localhost:8080/azure",
    api_key="test",
    api_version="test",
)

```

In [16]:
# azureClient = AzureOpenAI(
#     base_url="http://localhost:8080/azure",
#     api_key="test",
#     api_version="test",
# )

# completion = azureClient.chat.completions.create(
#     model="gpt-3.5-turbo",
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {"role": "user", "content": "Hello!"},
#     ],
# )
# print(completion)

### Boto3 Bedrock Runtime Client

```python
def _replace_headers(request: AWSRequest, **kwargs):
    request.headers = {"Authorization": "Bearer test"}

client = boto3.client("bedrock-runtime", endpoint_url="http://localhost:8080/bedrock")
client.meta.events.register("before-send.bedrock-runtime.*", _replace_headers)
```

In [6]:
import boto3
from botocore.exceptions import ClientError


def _replace_headers(request, **kwargs):
    request.headers = {"Authorization": "Bearer test"}


bedrockClient = boto3.client(
    "bedrock-runtime", endpoint_url="http://localhost:8080/bedrock"
)
bedrockClient.meta.events.register("before-send.bedrock-runtime.*", _replace_headers)

model_id = "us.meta.llama3-2-1b-instruct-v1:0"
user_message = "Say 'hi'"
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    # Send the message to the model, using a basic inference configuration.
    response = bedrockClient.converse(
        modelId=model_id,
        messages=conversation,
        inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
    )
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)

Hi


failed to send, dropping 2 traces to intake at http://localhost:8126/v0.5/traces after 3 retries


### Google Vertex Client

```python
import vertexai
from vertexai.preview.generative_models import GenerativeModel

vertexai.init(
    project=os.getenv("VERTEX_AI_PROJECT_ID"),
    api_endpoint="http://localhost:8080/vertex",
)

generative_multimodal_model = GenerativeModel("gemini-1.5-flash-002")
response = generative_multimodal_model.generate_content(["Say hi"])

print(response)
```

In [2]:
import os
import vertexai
from vertexai.preview.generative_models import GenerativeModel

vertexai.init(
    project=os.getenv("VERTEX_AI_PROJECT_ID", "encrypted-llm"),
    api_endpoint="http://0.0.0.0:8080/vertex",
    api_transport="rest",
)

generative_multimodal_model = GenerativeModel("gemini-2.0-flash-thinking-exp-01-21")
response = generative_multimodal_model.generate_content(["Say hi"])

print(response)


response = generative_multimodal_model.generate_content(
    ["WRite me a long story"], stream=True
)
for i in response:
    print(i)

failed to send, dropping 1 traces to intake at http://localhost:8126/v0.5/traces after 3 retries, 2 additional messages skipped


candidates {
  content {
    role: "model"
    parts {
      text: "Hi there! 😊 How can I help you today?"
    }
  }
  finish_reason: STOP
}
usage_metadata {
  prompt_token_count: 2
  candidates_token_count: 11
  total_token_count: 13
}
model_version: "gemini-2.0-flash-thinking-exp-01-21"

candidates {
  content {
    role: "model"
    parts {
      text: "The old lighthouse keeper, Silas, had seen a century of storms lash against the craggy coast of Aethelgard.  His beard, the color of sea foam and spun silver, reached his chest, and his eyes, the faded blue of a winter sky, held"
    }
  }
}
usage_metadata {
}
model_version: "gemini-2.0-flash-thinking-exp-01-21"

candidates {
  content {
    role: "model"
    parts {
      text: " the weight of countless sunrises and shipwrecks.  He’d watched generations of families come and go from the small, wind-bitten village nestled at the foot of the cliffs, each life a fleeting flicker against the enduring backdrop of the sea and the stone. Bu

### ELL Framework Example

In [ ]:
import ell
from pydantic import Field
import requests
from bs4 import BeautifulSoup

ell.init(verbose=True)


"""
BEDROCK CLIENT
"""


@ell.simple(model="us.meta.llama3-2-1b-instruct-v1:0", client=bedrockClient)
def bedrock_chat(prompt: str) -> str:
    return prompt


print(f'Bedrock \n\n {bedrock_chat("Hello, how are you?")}\n\n')

In [ ]:
"""
OpenAI Client
"""

@ell.simple(model="gpt-4o-mini", max_tokens=10, client=client)
def openai_client(prompt: str) -> str:
    return prompt


print(f'OpenAI \n\n {openai_client("Hello, how are you?")}\n\n')

In [ ]:
"""
TOOL USAGE
"""

@ell.tool()
def get_html_content(
    url: str = Field(
        description="The URL to get the HTML content of. Never include the protocol (like http:// or https://)"
    ),
):
    """Get the HTML content of a URL."""
    response = requests.get("https://" + url)
    soup = BeautifulSoup(response.text, "html.parser")
    return soup.get_text()[:100]


# OpenAI Client
@ell.complex(
    model="gpt-4o-mini",
    tools=[get_html_content],
    client=client,
)
def openai_get_website_content(website: str) -> str:
    return f"Tell me what's on {website}"


print("OpenAI Client Tool Use\n\n")
output = openai_get_website_content("new york times front page")
if output.tool_calls:
    print(output.tool_calls[0]())


# Bedrock Client
@ell.complex(
    model="anthropic.claude-3-haiku-20240307-v1:0",
    tools=[get_html_content],
    client=bedrockClient,
)
def bedrock_get_website_content(website: str) -> str:
    """You are an agent that can summarize the contents of a website."""
    return f"Tell me what's on {website}"


print("\n\nBedrock Client Tool Use\n\n")
output = bedrock_get_website_content("new york times front page")
if output.tool_calls:
    print(output.tool_calls[0]())

In [ ]:
from openai import OpenAI, AzureOpenAI

client = OpenAI(
    base_url="http://localhost:8080/openai-proxy/v1",
    api_key="test",
)
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Whats up dog?",
        }
    ],
    model="bedrock/us.anthropic.claude-3-haiku-20240307-v1:0",
    stream=False,
)

print(chat_completion)
print(chat_completion.choices[0].message.content)

In [ ]:
from openai import OpenAI, AzureOpenAI

client = OpenAI(
    base_url="http://localhost:8080/openai-proxy/v1",
    api_key="test",
)
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Write me a pretty short story",
        }
    ],
    model="bedrock/us.anthropic.claude-3-sonnet-20240307-v2:0",
    stream=True,
)

# for i in chat_completion:
#     print(i)
# print()


for i in chat_completion:
    print(i.choices[0].delta.content, end="")
print()

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID.",
                    },
                },
                "required": ["order_id"],
                "additionalProperties": False,
            },
        },
    }
]

messages = [
    {"role": "user", "content": "Hi, can you tell me the delivery date for my order?"}
]


client = OpenAI(
    base_url="http://localhost:8080/openai-proxy/v1",
    api_key="test",
)
# chat_completion = client.chat.completions.create(
#     model="bedrock/us.anthropic.claude-3-haiku-20240307-v1:0",
#     messages=messages,
#     # tools=tools,
#     stream=False
# )
chat_completion = client.chat.completions.create(
    messages=messages,
    model="bedrock/us.anthropic.claude-3-haiku-20240307-v1:0",
    stream=False,
    tools=tools,
    tool_choice="required",
)
print(chat_completion.choices[0])

In [ ]:
print(client.models.list())